In [1]:
import os
import random
from glob import glob

import helper
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil
import cv2

random.seed(2022)
np.random.seed(2022)

In [2]:
BASE_DIR = "marvel"
EXPORT_DIR = "dataset"

subsets = []
labels = []
images = []

for subset_dir in glob(os.path.join(BASE_DIR, "*")):
    subset = subset_dir.split("/")[-1]
    if not subset in ["train", "valid"]:
        continue

    for label_dir in glob(os.path.join(subset_dir, "*")):
        label = label_dir.split("/")[-1]
        for image_dir in glob(os.path.join(label_dir, "*")):
            subsets.append(subset)
            labels.append(label)
            images.append(image_dir)
metadata = pd.DataFrame(
    {"Original Image Path": images, "Label": labels, "Original Subset": subsets}
)

metadata_train = metadata[metadata["Original Subset"] == "train"]
metadata_train["Subset"] = "train"
metadata_test = metadata[metadata["Original Subset"] == "valid"]
metadata_test["Subset"] = "test"

metadata_train, metadata_valid = train_test_split(
    metadata_train, test_size=0.2, stratify=metadata_train["Label"]
)
metadata_valid["Subset"] = "valid"

metadata = pd.concat([metadata_train, metadata_valid, metadata_test])
metadata["Image Path"] = metadata[["Subset", "Original Image Path"]].apply(
    lambda row: os.path.join(EXPORT_DIR, row[0], "/".join(row[1].split("/")[2:])),
    axis=1,
)

metadata.to_csv('dataset.csv', index=False)

metadata

/tmp/ipykernel_9623/1159045677.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_train["Subset"] = "train"
/tmp/ipykernel_9623/1159045677.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_test["Subset"] = "test"


,Original Image Path,Label,Original Subset,Subset,Image Path
2352,marvel/train/black widow/pic_290.jpg,black widow,train,train,dataset/train/black widow/pic_290.jpg
1690,marvel/train/loki/pic_060.jpg,loki,train,train,dataset/train/loki/pic_060.jpg
814,marvel/train/thanos/pic_139.jpg,thanos,train,train,dataset/train/thanos/pic_139.jpg
1668,marvel/train/loki/pic_162.jpg,loki,train,train,dataset/train/loki/pic_162.jpg
1004,marvel/train/hulk/pic_077.jpg,hulk,train,train,dataset/train/hulk/pic_077.jpg
...,...,...,...,...,...
3030,marvel/valid/black widow/pic_094.jpg,black widow,valid,test,dataset/test/black widow/pic_094.jpg
3031,marvel/valid/black widow/pic_296.jpg,black widow,valid,test,dataset/test/black widow/pic_296.jpg
3032,marvel/valid/black widow/pic_093.jpg,black widow,valid,test,dataset/test/black widow/pic_093.jpg
3033,marvel/valid/black widow/pic_264.jpg,black widow,valid,test,dataset/test/black widow/pic_264.jpg


In [3]:
shutil.rmtree('dataset', ignore_errors=True)
for row in metadata.iloc:
    folder = '/'.join(row['Image Path'].split('/')[:-1])
    os.makedirs(folder, exist_ok=True)
    image = helper.load_image(row['Original Image Path'])
    helper.save_image(image, row['Image Path'])